In [1]:
import warnings
import pandas as pd
import numpy as np
!pip install yfinance

import yfinance as yf
import datetime as dt
from pandas_datareader import data as pdr
import matplotlib.pyplot as plt
import plotly.graph_objs as go

yf.pdr_override()


     |████████████████████████████████| 5.5MB 6.9MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=336a20d8267ec289ee6e65b6640d1340a3184e701815fd7d7ed1f3d9ba402947
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
startyear = 2005
startmonth = 1
startday = 1

start = dt.datetime(startyear, startmonth, startday)
now = dt.datetime.now()

fee = 0.98

In [3]:
class Investor:
    def __init__(self, income, strategy, capital=0):
        self.income = income 
        self.strategy = strategy
        self.capital = capital
        self.portfolio = dict()
        self.df = self.strategy.df 
        self.baseline = [] 
        self.capital_history = []
        self.simulate()
    
    def _buy(self, stock, price, consideration):
        self.capital -= consideration
        
        units = consideration * fee / price

        if len(self.portfolio[stock]) != 0 :
            self.portfolio[stock].append(self.portfolio[stock][-1] + units)
        else:
            self.portfolio[stock].append(units)
                           
    def _sell(self, stock, price, consideration):
        self.capital += consideration 
        units = consideration / price 
        self.portfolio[stock].append(self.portfolio[stock][-1] - units/fee)

    def _pass(self, stock):
        if len(self.portfolio[stock]) == 0:
            self.portfolio[stock].append(0)
        else:
            self.portfolio[stock].append(self.portfolio[stock][-1])

    def action(self, date, months):
        signal, prices = self.strategy.assess(date, self.capital, self.portfolio, months)
        for stock, consideration in signal.items():
            if consideration > 0:
                self._buy(stock, prices[stock], consideration)
            elif consideration < 0:
                self._sell(stock, prices[stock], -consideration)
            else:
                self._pass(stock)
        #print(self.capital)
        #assert np.isclose(self.capital, 0)

    def simulate(self):
        for stock in self.strategy.stocks:
            self.portfolio[stock] = []
        
        months = []
        for date in self.df.Date:
            if len(months) == 0 or months[-1] != (date.month, date.year):
                self.capital += self.income
                try:
                    self.baseline.append(self.baseline[-1]+self.income)
                except IndexError:
                    self.baseline.append(self.income)
            else:
                try:
                    self.baseline.append(self.baseline[-1])
                except:
                    self.baseline.append(0)
                
            self.action(date, months)
            months.append((date.month, date.year))
            self.capital_history.append(self.capital)
        
    
    @property
    def history(self):
        df_val = self.df.copy()[self.strategy.stocks]
        for stock, units in self.portfolio.items():
            df_val[stock] *= units
        return df_val.sum(axis=1) + np.array(self.capital_history)

In [4]:
class Strategy:
    def __init__(self, stocks):
        self.stocks = stocks
        self.df = self.prepare_df()
    
    def prepare_df(self):
        global start
        global now 
        date = None 
        prices = dict() 
        for stock in self.stocks:
            df = pdr.get_data_yahoo(stock, start, now)
            if date is None: 
                date = list(df.index) 
            price = df.Close.tolist()
            prices[stock] = price
        df = pd.DataFrame(date, columns=['Date'])
        for stock, price in prices.items():
            df[stock] = price
        return df 

    def query_price(self, date):
        price_dict = dict()
        for stock in self.stocks:
            price = self.df[self.df.Date == date][stock].values
            price_dict[stock] = float(price)
        return price_dict

    def assess(self, *args):        
        pass

class DCA(Strategy):
    def __init__(self, stocks, weights, rebalance_every=False):
        super().__init__(stocks)
        assert len(stocks) == len(weights)
        weights = [i/sum(weights) for i in weights]
        self.weighting = dict(zip(self.stocks, weights))
        self.rebalance_every = rebalance_every

    def assess(self, date, capital, portfolio, months):
        prices = self.query_price(date)
        #print(prices)
        signals = self.weighting.copy()
        #print(capital)
        if self.rebalance_every is not False:
            rebalance = len(set(months)) % self.rebalance_every == 0
        else:
            rebalance = False
        if not rebalance or len(months) == 0:
            for i in signals:
                if capital > 0:
                    signals[i] *= capital
                else:
                    signals[i] = 0
        else: 
            if capital > 0:
                value_dict = dict()
                for stock, price in prices.items():
                    value = portfolio[stock][-1] * price
                    value_dict[stock] = value
                    #print(f'Sigs{signals} {stock}Portfolio{portfolio[stock][-1]}')

                total_value = sum(value_dict.values()) + capital
                #print(total_value)
                signals = {stock: w*total_value - value_dict[stock] for stock, w in signals.items()}
            else:
                signals = {stock: 0 for stock, w in signals.items()}
        #print(signals)
        return signals, prices

class EMACrossover(Strategy):
    def __init__(self, stocks, weights, active_trading_percent, window_short, window_long):
        super().__init__(stocks)
        weights = [i/sum(weights) for i in weights]
        self.weighting = dict(zip(self.stocks, weights))
        self.active_trading_percent = active_trading_percent
        self.window_short = window_short
        self.window_long = window_long
        self.modify_df()

    def modify_df(self):
        for stock in self.stocks:
            price = self.df[stock]
            ema_short = self.generateEMA(price, self.window_short)
            ema_long = self.generateEMA(price, self.window_long)
            self.df[f'{stock}_EMA_short'] = ema_short
            self.df[f'{stock}_EMA_long'] = ema_long
            self.df[f'{stock}_signals'] = self.generateSignals(self.df[f'{stock}_EMA_short'], self.df[f'{stock}_EMA_long'])
                                         
    def query_signals(self, date):
        signals_dict = dict()
        for stock in self.stocks:
            signal = self.df[self.df.Date == date][f'{stock}_signals'].values
            signals_dict[stock] = signal
        return signals_dict


    @staticmethod
    def generateSignals(short, long):
        signals = []
        comparison =  list(short > long)
        for idx in range(len(comparison)):
            signals.append(None)
            if idx > 3:
                window = comparison[idx-3:idx]
                if window[0] == False and window[1] == True and window[-1] == True:
                    signals[-1] = True
                if window[0] == True and window[1] == False and window[-1] == False and  [i for i in signals if i is not None][-1] == True:
                    signals[-1] = False
        return signals

    @staticmethod
    def generateEMA(price, window_size):
        multiplier = 2 / (window_size + 1 )
        EMA = []
        for i in range(len(price)):
            if i == window_size:
                sma = sum(price[i-window_size:i])/window_size
                EMA.append(sma)
            elif i >window_size:
                ema = price[i] * multiplier + EMA20[-1] * (1-multiplier)
                EMA.append(ema)
            else: 
                EMA.append(price[i])
        return EMA


    def assess(self, date, capital, portfolio, months):
        prices = self.query_price(date)
        ema_signals = self.query_signals(date)
        signals = self.weighting.copy()
        
        for stock, s in ema_signals.items(): 
            if s == True:
                signals[stock] *= capital
            elif s == False:  # sell 20% of existing stock value 
                portfolio_value = portfolio[stock][-1] * prices[stock]
                signals[stock] = -0.2 * portfolio_value
            else:
                signals[stock] = 0 

        return signals, prices


In [5]:
# investor = Investor(income=1000, strategy=EMACrossover(['SPY'], [100],window_short=50, window_long=100, active_trading_percent=0.2))

In [6]:
investor1 = Investor(income=1000, strategy=DCA(['SPY'], [100]))

investor2 = Investor(income=1000, strategy=DCA(['SPY', 'GLD'], [80, 20]))

investor3 = Investor(income=1000, strategy=DCA(['SPY', 'GLD'], [80, 20], rebalance_every=2))
investor4 = Investor(income=1000, strategy=DCA(['SPY', 'GLD'], [80, 20], rebalance_every=6))

investor5 = Investor(income=1000, strategy=DCA(['SPY', 'GLD'], [50, 50]))

investor6 = Investor(income=1000, strategy=DCA(['SPY', 'GLD'], [50, 50], rebalance_every=2))
investor7 = Investor(income=1000, strategy=DCA(['SPY', 'GLD'], [50, 50], rebalance_every=6))



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
index = investor1.df.Date
baseline = investor1.baseline

fig = go.Figure(layout_title_text=f"{start.month}/{start.year}-{now.month}/{now.year}")
fig.add_trace(go.Scatter(x=index, y=baseline, name='Baseline'))
# fig.add_trace(go.Scatter(x=index, y=investor.history, name='Active Trading'))
fig.add_trace(go.Scatter(x=index, y=investor1.history, name='SPY only'))
fig.add_trace(go.Scatter(x=index, y=investor2.history, name='80-20 no rebalance'))
fig.add_trace(go.Scatter(x=index, y=investor3.history, name='80-20 2m rebalance'))
fig.add_trace(go.Scatter(x=index, y=investor4.history, name='80-20 6m rebalance'))
fig.add_trace(go.Scatter(x=index, y=investor5.history, name='50-50 no rebalance'))
fig.add_trace(go.Scatter(x=index, y=investor6.history, name='50-50 2m rebalance'))
fig.add_trace(go.Scatter(x=index, y=investor7.history, name='50-50 6m rebalance'))


fig.update_xaxes(title_text='Years')
fig.update_yaxes(title_text='Value')